In [1]:
# import
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import random
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from IPython import display
import time
from pathlib import Path
import pandas as pd
import os
import copy
import networkx as nx
import pickle
import sys
from collections import Counter
from scipy import stats
%matplotlib inline

# change directory

os.chdir("/home/kyoko/Develop/ActualCausation/MABE/MABE_contents/actual_agency")

# import packages -- pyphi

from pyanimats import *
from pyTPM import *
import actual_agency as agency
import pyphi
from pyphi import actual, config, Direction

pyphi.config.VALIDATE_SUBSYSTEM_STATES = False
pyphi.config.PARTITION_TYPE = 'TRI'

# Change directory back to Example directory, now in MABE_contents
os.chdir("../Experiments/analysis_now")

%load_ext autoreload
%autoreload 2
%aimport pyTPM, pyanimats, actual_agency

# make an array
datas = np.zeros(3000).reshape(250, 12)

prearr = np.sort(np.tile(np.arange(1,11), (1,5))).reshape(50, 1) 
runarr = np.tile(prearr.copy(), (5,1)) #Runs
hidarr = np.sort(np.tile(np.arange(2,7), (1,50))).reshape(250, 1) #Hidden nodes
stsarr = np.tile(np.arange(1,6), (1,50)).reshape(250, 1)

datas[:,0:1] = hidarr
datas[:,1:2] = runarr
datas[:,2:3] = stsarr

# defining some parameters to use 
generations = 30000
n_agents = int(generations/500.+1)
n_runs = 10
n_trials = 64
n_data = 5

agent = 60 # saved generation
trial = 60 # 0~63
run = 1 # 0 ~ n_runs-1 change only this


# Get Line-of-descent (LOD) data from MABE output
    
LOD_data = ['190822_7156331_h2_LOD_data.pkl','190822_7156331_h3_LOD_data.pkl','190822_7156331_h4_LOD_data.pkl','190822_7156331_h5_LOD_data.pkl','190822_7156331_h6_LOD_data.pkl']
LODs = []    
for i in range(len(LOD_data)):
    with open(LOD_data[i],'rb') as f:
        LODs.append(pickle.load(f))

# Get genome data
all_genomes = ['190822_7156331_h2_genome.pkl','190822_7156331_h3_genome.pkl','190822_7156331_h4_genome.pkl','190822_7156331_h5_genome.pkl','190822_7156331_h6_genome.pkl']
genomes = []
for i in range(len(all_genomes)):
    with open(all_genomes[i],'rb') as f:
        genomes.append(pickle.load(f))


# Get activity data
all_activity = ['190822_7156331_h2_activity.pkl','190822_7156331_h3_activity.pkl','190822_7156331_h4_activity.pkl','190822_7156331_h5_activity.pkl','190822_7156331_h6_activity.pkl']
activities = []
for i in range(len(all_activity)):
    with open(all_activity[i],'rb') as f:
        activities.append(pickle.load(f))


# Define functions to find states
# find the most frequent state


def counting3(data):
    c = Counter(data)
    freq_scores = c.most_common()
    if len(freq_scores) < 3:
        return(freq_scores)
    else:
        third_count = freq_scores[2][1]
    
        modes = [freq_scores[0][:], freq_scores[1][:]]
        for p in range(2,len(freq_scores)): #出現回数を比較、追加
            if freq_scores[p][1] == third_count and p < 5:
                modes.append(freq_scores[p])
        return(modes)

# change num(10) to bin(2)
def int2binary(n,node_n):
    holi = [int(x) for x in bin(n)[2:].zfill(node_n)]
    holi.reverse()
    return holi


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [2]:
def num2state(num,n_nodes):
 
    number = '{0:0' + str(n_nodes) + 'b}'
    state = number.format(num)
    state = [int(i) for i in state]

    # returns the state
    return state

In [3]:
# brain acivity, genome, TPM, TPM_gates, cm
genome = []
TPM, TPM_gates, cm = [],[],[]
n_connection = np.zeros(n_data*n_runs)
brain_activity = []

for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
        
        # culculate brain activity
        brain_activity.append(agency.getBrainActivity(activities[data][n],n_agents=n_agents, n_nodes=n_hnodes+4, n_hidden=n_hnodes))
        #actarr = brain_activity[place][agent][trial]


Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating a

In [4]:

for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
        genome.append(agency.get_genome(genomes[data], n, agent))
#print(len(genome[4]))

        a, b, c = genome2TPM(genome[place], n_nodes=n_hnodes+4, n_sensors=2, n_motors=2, gate_type='deterministic',states_convention='loli',remove_sensor_motor_effects=True)
        TPM.append(a)
        TPM_gates.append(b)
        cm.append(c) # right?
        

In [5]:
prestate = np.array([[[36, 16, 4], [20,56], [36,8], [0,16,48,40], [32,0,36], [40,56,14,41],[52,12],[5,24,12], [0,20], [0,52]],
[[112,0,56,24], [0,24,18,70], [84,44,68], [60,12,120,72,121],[32,52,60], [53,100,48,112,40], [48,49,97,107,72], [44,4],[84,108], [32,8,40]],
 [[184,92,93,235,191], [216,92,217,125,111], [212,76,92,196], [32,8,2], [144,146,145,129,163], [172,228,236], [236,220,8,79,204], [212, 96],[44,80,96], [4]],
 [[16,272,388], [192,456,462,198], [376,371,370], [24,8,168,68], [36], [404,220,144], [100], [0,328,288,228], [21,28,72], [12,296,280]],
 [[516], [164,416,484], [608,628,540,520], [708,136], [560,561,563,547,610], [556,564,812], [156,664,728], [80,372,244,116,16], [256,272,464], [244, 112, 372]]])

In [34]:
prestate[0][0]

[36, 16, 4]

In [32]:
state = []
for i in range(len(prestate)):
    for j in range(len(prestate[i])):
        for k in range(len(prestate[i][j])):
            state.append(np.array(num2state(prestate[i][j][k], 6+i)))
state = np.array(state)
state

array([array([1, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0]),
       array([0, 0, 0, 1, 0, 0]), array([0, 1, 0, 1, 0, 0]),
       array([1, 1, 1, 0, 0, 0]), array([1, 0, 0, 1, 0, 0]),
       array([0, 0, 1, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]),
       array([0, 1, 0, 0, 0, 0]), array([1, 1, 0, 0, 0, 0]),
       array([1, 0, 1, 0, 0, 0]), array([1, 0, 0, 0, 0, 0]),
       array([0, 0, 0, 0, 0, 0]), array([1, 0, 0, 1, 0, 0]),
       array([1, 0, 1, 0, 0, 0]), array([1, 1, 1, 0, 0, 0]),
       array([0, 0, 1, 1, 1, 0]), array([1, 0, 1, 0, 0, 1]),
       array([1, 1, 0, 1, 0, 0]), array([0, 0, 1, 1, 0, 0]),
       array([0, 0, 0, 1, 0, 1]), array([0, 1, 1, 0, 0, 0]),
       array([0, 0, 1, 1, 0, 0]), array([0, 0, 0, 0, 0, 0]),
       array([0, 1, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0]),
       array([1, 1, 0, 1, 0, 0]), array([1, 1, 1, 0, 0, 0, 0]),
       array([0, 0, 0, 0, 0, 0, 0]), array([0, 1, 1, 1, 0, 0, 0]),
       array([0, 0, 1, 1, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0]),
       ar

In [27]:
cm = np.array(cm)
for i in range(50):
    print(pyphi.connectivity.is_strong(cm[i][4:,4:]))

False
False
False
True
False
True
True
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False


In [52]:
complexes = pyphi.compute.network.possible_complexes(network, sta)

NameError: name 'sta' is not defined

In [ ]:
comps =list(complexes)
comps

In [45]:
num_connected = []
MC = []
count = 0

for i in range(50):
    network = pyphi.network.Network(TPM[i],cm[i])
    #sta = tuple(cm[i][0,:].astype(int))
    row = int(i/10)
    cul = i%10
    print(row, cul)
    for j in range(len(prestate[row][cul])):
        print(state[count+j])
        MC.append(pyphi.compute.network.major_complex(network,state[count+j]))
        num_connected.append(len(list(pyphi.compute.network.possible_complexes(network, state[count+j]))))
    count = count+j+1 
    print(j, count)

Computing concepts:   0%|          | 0/3 [00:00<?, ?it/s]

0 0
[1 0 0 1 0 0]
[0 1 0 0 0 0]
[0 0 0 1 0 0]
2 3
0 1
[0 1 0 1 0 0]
[1 1 1 0 0 0]
1 5
0 2
[1 0 0 1 0 0]
[0 0 1 0 0 0]
1 7
0 3
[0 0 0 0 0 0]



Computing concepts:   0%|          | 0/3 [00:00<?, ?it/s]

[0 1 0 0 0 0]



Computing concepts:   0%|          | 0/3 [00:00<?, ?it/s]

...

[0 0 0 1 1 1 0 0 0 0]



Computing concepts:  71%|███████▏  | 5/7 [00:00<00:00,  6.46it/s]
                                                                 
Computing concepts:  33%|███▎      | 1/3 [00:00<00:00,  6.51it/s]
                                                                 
Computing concepts:  33%|███▎      | 1/3 [00:00<00:00,  6.48it/s]
                                                                 
Computing concepts:   0%|          | 0/7 [00:00<?, ?it/s]

[0 1 0 1 1 1 0 1 0 0]



Computing concepts:  71%|███████▏  | 5/7 [00:00<00:00,  6.71it/s]
                                                                 
Computing concepts:  33%|███▎      | 1/3 [00:00<00:00,  6.03it/s]
                                                                 
Computing concepts:  33%|███▎      | 1/3 [00:00<00:00,  6.64it/s]
                                                                 
Evaluating Φ cuts:  50%|█████     | 1/2 [00:00<00:00,  2.54it/s]
                                                                

2 160


In [50]:
len(num_connected)

160

In [56]:
prestate[0][0]

[36, 16, 4]

In [71]:
alldicts=[]
counter = 0
data_counter = 0
for d in prestate:
    run_counter = 0
    for r in d:
        state_counter = 0
        for s in range(len(r)):
            newdict = {'run' : run_counter,
                       'data' : data_counter,
                       'state' : s,
                       'Phi' : MC[counter].phi
                      }
            alldicts.append(newdict)
            counter+=1
        run_counter+=1
    data_counter+=1

In [74]:
print([d for d in alldicts if d['Phi']>0])

[{'run': 7, 'data': 0, 'state': 1, 'Phi': 1.0}, {'run': 4, 'data': 1, 'state': 0, 'Phi': 0.875}, {'run': 4, 'data': 1, 'state': 1, 'Phi': 0.484376}, {'run': 4, 'data': 1, 'state': 2, 'Phi': 0.484376}, {'run': 5, 'data': 1, 'state': 0, 'Phi': 0.687499}, {'run': 5, 'data': 1, 'state': 2, 'Phi': 0.381943}, {'run': 5, 'data': 1, 'state': 4, 'Phi': 0.381943}, {'run': 8, 'data': 1, 'state': 0, 'Phi': 0.381943}, {'run': 8, 'data': 1, 'state': 1, 'Phi': 0.381943}, {'run': 0, 'data': 2, 'state': 1, 'Phi': 1.0}, {'run': 0, 'data': 2, 'state': 2, 'Phi': 1.0}, {'run': 0, 'data': 3, 'state': 2, 'Phi': 1.0}, {'run': 1, 'data': 3, 'state': 0, 'Phi': 2.812499}, {'run': 1, 'data': 3, 'state': 1, 'Phi': 0.875}, {'run': 1, 'data': 3, 'state': 2, 'Phi': 1.14583}, {'run': 1, 'data': 3, 'state': 3, 'Phi': 0.58333}, {'run': 3, 'data': 3, 'state': 0, 'Phi': 0.333333}, {'run': 3, 'data': 3, 'state': 1, 'Phi': 0.6875}, {'run': 3, 'data': 3, 'state': 2, 'Phi': 0.472656}, {'run': 3, 'data': 3, 'state': 3, 'Phi': 

In [68]:
counter = 0
for d in prestate:
    for r in d:
        for s in range(len(r)):
            print(counter)
            counter+=1
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


In [61]:
alldicts

{'run': 1, 'data': 1, 'state': 1, 'Phi': 0.0}

In [62]:
counter

1

In [53]:
pyphi.compute.subsystem.ces()

NameError: name 'ces' is not defined

In [ ]:
num_connected

In [ ]:
# def _test(arr):
#     mode = arr**2
#     return mode

# array = [[3, 14, 1],[2,5,6]]
# test = np.frompyfunc(_test, 1, 1)

# a = test(array)
# a

In [ ]:
# # 一つとってくる
# # 二進数に変換
# # 文字ごとに分割
# # 0で埋める
# # tupleにする？



# def _InttoState(arr, x, y):
#     mode = bin(arr[x,y])[:2]
#     a = list(mode)
#     for i in range():
#         a[i] = int(a[i])
#     return a


# xarr = np.arange(5)
# yarr = np.arange(10)

# InttoState = np.frompyfunc(_InttoState, 3, 1)

# InttoState(state, xarr, yarr)

# # InttoState = np.frompyfunc(_InttoState, 4, 1)
# # InttoState(state,1)

In [ ]:
# arr = 42
# mode = bin(state[0][0][1])[2:]
# a = list(mode)
# for i in range(8):
#     a[i] = int(a[i])
#     if len(a) < 8:
#         a.insert(0,0)
        
# tuple(a)

In [ ]:
# error
# for i in range(5):
#     for j in range(10):
#         for k in range(5):
#             if state[i][j][k:k+1]:
#                 mode = np.array(bin(arr[i][j][k])[:2])
#                 a = list(mode)
#                 for i in range():
#                     a[i] = int(a[i])
#                 else:
#                     break